## Ball Picking Challenge

### Part 1: Navigation

The aim is to program the robot for the first part of the challenge: follow the line from the beginning to the destination point. To do so, you need to reuse the abilities learnt in weeks 1-3; please feel free to reuse the code of those notebooks and exercises.

### Initialization

In [9]:
from packages import initialization
from packages import pioneer3dx as p3dx
import matplotlib.pyplot as plt
import numpy
import cv2
p3dx.init()

Found ROS controller /pioneer3dx_11552_ip_172_31_2_203


### Help functions

In [10]:
def color_blob(lower, upper):
    hsv = cv2.cvtColor(p3dx.image, cv2.COLOR_RGB2HSV)
    mask = cv2.inRange(hsv, lower, upper)
    M = cv2.moments(mask)
    area = M['m00']
    if area > 0:
        cx = int(M['m10']/area)
        cy = int(M['m01']/area)
    else:
        cx = None
        cy = None
    return area, cx, cy

In [11]:
def move(V_robot,w_robot):
    r = 0.1953 / 2
    L = 0.33
    w_r = (2 * V_robot + L * w_robot) / (2*r)
    w_l = (2 * V_robot - L * w_robot) / (2*r)
    p3dx.move(w_l, w_r)

In [12]:
# Lower-level functions
def line_centroid(image):
    hsv = cv2.cvtColor(p3dx.image, cv2.COLOR_RGB2HSV)
    lower_violet = numpy.array([140, 100, 80])
    upper_violet = numpy.array([160, 255, 255])
    mask = cv2.inRange(hsv, lower_violet, upper_violet)
    mask[0:80, 0:150] = 0
    M = cv2.moments(mask)
    if int(M['m00']) is not 0:
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
    else:
        cx = 0
        cy = 0
    return cx, cy

In [13]:
# Lower-level functions
def green_centroid():
    hsv = cv2.cvtColor(p3dx.image, cv2.COLOR_RGB2HSV)
    lower = numpy.array([50, 100, 80])
    upper = numpy.array([70, 255, 255]) 
    mask = cv2.inRange(hsv, lower, upper)
    mask[0:80, 0:150] = 0
    M = cv2.moments(mask)
    area = M['m00']
    if int(M['m00']) is not 0:
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
    else:
        cx = 0
        cy = 0
    return area, cx, cy

In [14]:
def follow_line():
    
    width = p3dx.image.shape[1]
    Kp = 0.1
    while is_green_detected()[1] < 300000:
        cx, cy = line_centroid(p3dx.image)
        #print (cx, is_green_detected())
        if is_green_detected()[0] and is_green_detected()[2] > cx:
            move(2, -2)
            #print("Green detected on right")
        
        elif is_green_detected()[0] and is_green_detected()[2] < cx:
            move(2, 2)
            #print("Green detected on left")
        
        elif cy == 0 :
            move(2, 0.5)
            #print("Hole detected")
        
        elif color_blob(numpy.array([ 0, 100, 100]), numpy.array([ 10, 255, 255]))[0] > 200000:
            #print("Red obstacle detected")
            R = 0.9
            V = 0.35
            w = V/R
            move(V,-w)
            p3dx.sleep(2.2)
            move(V,w*1.5)
            p3dx.sleep(2.7)
            move(V,-w*4)
            p3dx.sleep(1)
    
        else:
            err = cx - (width/2)
            linear = 2 #The linear velocity is constant, e.g. 2m/s
            angular = - Kp * err # the angular velocity w
            move(linear, angular)
            #print('Following the line')
    move(0,0)
    #print('Large green detected')

In [ ]:
def is_green_detected():
    area, cx, cy = green_centroid()
    if area == 0:
        return False, area, cx, cy
    else:
        return True, area, cx, cy

### Main Loop

In [ ]:
p3dx.tilt(-0.45)
try:
    while True:
        follow_line()    
except KeyboardInterrupt:
    move(0,0)

### Check the image

In [ ]:
%matplotlib inline
plt.imshow(p3dx.image);

In [ ]:
lower = numpy.array([50, 100, 80])
upper = numpy.array([70, 255, 255]) 
hsv = cv2.cvtColor(p3dx.image, cv2.COLOR_RGB2HSV)
mask = cv2.inRange(hsv, lower, upper)
#mask[0:80, 0:150] = 0
M = cv2.moments(mask)
area = M['m00']
plt.imshow(mask,cmap='gray')
print(area)

---
#### Try-a-Bot: an open source guide for robot programming
Developed by:
[![Robotic Intelligence Lab @ UJI](img/logo/robinlab.png "Robotic Intelligence Lab @ UJI")](http://robinlab.uji.es)

Sponsored by:
<table>
<tr>
<td style="border:1px solid #ffffff ;">
<a href="http://www.ieee-ras.org"><img src="img/logo/ras.png"></a>
</td>
<td style="border:1px solid #ffffff ;">
<a href="http://www.cyberbotics.com"><img src="img/logo/cyberbotics.png"></a>
</td>
<td style="border:1px solid #ffffff ;">
<a href="http://www.theconstructsim.com"><img src="img/logo/theconstruct.png"></a>
</td>
</tr>
</table>

Follow us:
<table>
<tr>
<td style="border:1px solid #ffffff ;">
<a href="https://www.facebook.com/RobotProgrammingNetwork"><img src="img/logo/facebook.png"></a>
</td>
<td style="border:1px solid #ffffff ;">
<a href="https://www.youtube.com/user/robotprogrammingnet"><img src="img/logo/youtube.png"></a>
</td>
</tr>
</table>